# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686603


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:07<01:44,  3.61s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:12,  2.59s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:47,  1.84s/it]

Rendering models:  23%|██▎       | 7/31 [00:07<00:31,  1.33s/it]

...

Rendering models:  84%|████████▍ | 26/31 [00:09<00:00,  6.54it/s]

Rendering models:  87%|████████▋ | 27/31 [00:10<00:00,  6.30it/s]

Rendering models:  94%|█████████▎| 29/31 [00:10<00:00,  7.33it/s]

Rendering models: 100%|██████████| 31/31 [00:10<00:00,  7.48it/s]

froboy                                0.006151
zac8151                               0.001008
jammiedodgers                         0.006250
shirogane                             0.002888
Dynamatt                              0.000680
Petrusperes                           0.048735
Staceyann116                          0.016847
shocko61                              0.000736
djswanso                              0.000754
kjkavene                              0.003794
theetick                              0.079941
sn346808                              0.004361
kaitybug                              0.004152
puic                                  0.000644
not-logged-in-2cac2eef728c432df1d3    0.052707
nsdeni                                0.001556
Mohit.Bhardwaj                        0.033754
cosmicdave                            0.000860
not-logged-in-5fb91835bfb82f1736a3    0.005254
not-logged-in-c4f439475ebe12310454    0.002607
ElisabethB                            0.003302
quinnbee22   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())